## AutoML with AutoKeras
resource: [Jeff Heaton](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class_14_01_automl.ipynb)

In [ ]:
import tensorflow as tf
import keras_preprocessing
import nuumpy as np
import os 
import glob
import tqdm
from PIL import Image

In [ ]:
!pip install autokeras

In [ ]:
import autokeras

#### Download and preprocess data

In [ ]:
URL = 'https://github.com/jeffheaton/data-mirror/'
DOWNLOAD_SOURCE = URL + 'releases/download/v1/paperclips.zip'
DOWNLOAD_NAME = DOWNLOAD_SOURCE[DOWNLOAD_SOURCE.rfind('/')+1:]

if COLAB:
    PATH = '/content'
else:
    PATH = '/data'
    
EXTRACT_TARGET = os.path.join(PATH, 'clips')
SOURCE = os.path.join(PATH, 'paperclips')

!wget -O {os.path.join(PATH,DOWNLOAD_NAME)} {DOWNLOAD_SOURCE}
!mkdir -p {SOURCE}
!mkdir -p {TARGET}
!mkdir -p {EXTRACT_TARGET}
!unzip -o -j -d {SOURCE} {os.path.join(PATH, DOWNLOAD_NAME)} >/dev/null

df_tain = pd.read_csv(os.path.join(SOURCE, 'train.csv'))
df_train['filename'] = 'clips-' + df_train.id.astype(str) + '.jpg'

df_train = df[0:1000]

In [ ]:
IMG_SHAPE = (128, 128)

def load_images(files, img_shape):
    count = len(files)
    x = np.zeros((count,) + img_shape + (3,))
    i = 0
    for file in tqdm.tqdm(files):
        img = Image.open(file)
        img = img.resize(img_shape)
        img = np.array(img)
        img = img/255.
        x[i,:,:,:] = img
        i+=1
    return x

images = [os.path.join(SOURCE, x) for x in df_train.filename]
x = load_images(images, IMG_SHAPE)
y = df.train.clip_count.values

#### Train with AutoKeras

In [ ]:
MAX_TRIALS = 2
SEED = 42
VAL_SPLIT = 0.1
EPOCHS = 1000
BATCH_SIZE = 32

In [ ]:
auto_reg = autokeras.ImageRegressor(overwrite=True, max_trails=MAX_TRIALS, seed=42)

auto_reg.fit(x, y, validation_split=VAL_SPLIT, batch_size=BATCH_SIZE, epochs=EPOCHS)

print(auto_reg.evaluate(x, y))

In [ ]:
model = auto_reg.export_model()
model.summary()

#### export model

In [ ]:
try:
    model.save('model_autokeras', save_format='tf')
except:
    model.save('model_autokeras.h5')
    
loaded_model = keras.models.load_model('model_autokeras', custom_object=autokeras.CUSTOM_OBJECTS)

print(loaded_model.evaluate(x, y))